In [63]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/ecommerce-dataset/item_properties_part1.csv
/kaggle/input/ecommerce-dataset/category_tree.csv
/kaggle/input/ecommerce-dataset/item_properties_part2.csv
/kaggle/input/ecommerce-dataset/events.csv


In [64]:
!pip install recbole

# 1. Create atomic file

In [65]:
import pandas as pd
import gc
df = pd.read_csv("../input/ecommerce-dataset/events.csv")
df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      int64  
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 105.1+ MB


In [67]:
# from datetime import date
import datetime
import calendar

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      int64  
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 105.1+ MB


we will work only on view event 


In [69]:
df = df[df["event"] == "view"]
df

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN
...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN
2756097,1438399813142,762376,view,115946,NaN
2756098,1438397820527,1251746,view,78144,NaN
2756099,1438398530703,1184451,view,283392,NaN


In [70]:
df["event"].value_counts()

view    2664312
Name: event, dtype: int64

We save atomic file in dataset format for using with recbole

In [71]:
!mkdir /kaggle/working/recbox_data

mkdir: cannot create directory ‘/kaggle/working/recbox_data’: File exists


In [72]:
df.to_csv('/kaggle/working/recbox_data/recbox_data.inter', index=False, sep='\t')

gc.collect()

93

### 3. Create dataset and train model with Recbole

In [73]:
import os
import numpy as np
import pandas as pd
gc.collect()

23

In [74]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

In [75]:
df.head(2)

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN


In [76]:
temp = df[['visitorid', 'itemid', 'timestamp']].rename(
    columns={'visitorid': 'user_id:token', 'itemid': 'item_id:token', 'timestamp': 'timestamp:float'})
temp

,user_id:token,item_id:token,timestamp:float
0,257597,355908,1433221332117
1,992329,248676,1433224214164
2,111016,318965,1433221999827
3,483717,253185,1433221955914
4,951259,367447,1433221337106
...,...,...,...
2756096,591435,261427,1438398785939
2756097,762376,115946,1438399813142
2756098,1251746,78144,1438397820527
2756099,1184451,283392,1438398530703


In [77]:
# !mkdir /kaggle/working/recbox_data
temp.to_csv('/kaggle/working/recbox_data/recbox_data.inter', index=False, sep='\t')
del temp
gc.collect()

0

In [78]:
parameter_dict = {
    'data_path': '/kaggle/working',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'user_inter_num_interval': "[10,inf)",
    'item_inter_num_interval': "[10,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'neg_sampling': None,
    'epochs': 10,
    'eval_args': {
        'split': {'RS': [.8, .1, .1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full',
    'metrics':'AUC'}
}

config = Config(model='GRU4Rec', dataset='recbox_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
logger.info(config)


General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = /kaggle/working/recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
neg_sampling = None
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full', 'metrics': 'AUC'}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATING_FIELD = rating
TIME_FIELD = ti

In [79]:
dataset = create_dataset(config)
logger.info(dataset)

recbox_data
The number of users: 11032
Average actions of users: 25.010153204605203
The number of items: 10375
Average actions of items: 26.59408135723925
The number of inters: 275887
The sparsity of the dataset: 99.75896013349993%
Remain Fields: ['user_id', 'item_id', 'timestamp']
recbox_data
The number of users: 11032
Average actions of users: 25.010153204605203
The number of items: 10375
Average actions of items: 26.59408135723925
The number of inters: 275887
The sparsity of the dataset: 99.75896013349993%
Remain Fields: ['user_id', 'item_id', 'timestamp']
recbox_data
The number of users: 11032
Average actions of users: 25.010153204605203
The number of items: 10375
Average actions of items: 26.59408135723925
The number of inters: 275887
The sparsity of the dataset: 99.75896013349993%
Remain Fields: ['user_id', 'item_id', 'timestamp']


In [80]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

[Training]: train_batch_size = [2048] negative sampling: [None]
[Training]: train_batch_size = [2048] negative sampling: [None]
[Training]: train_batch_size = [2048] negative sampling: [None]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full', 'metrics': 'AUC'}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full', 'metrics': 'AUC'}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full', 'metrics': 'AUC'}]


In [81]:
# model loading and initialization
model = GRU4Rec(config, train_data.dataset).to(config['device'])
logger.info(model)

# trainer loading and initialization
trainer = Trainer(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data)

GRU4Rec(
  (item_embedding): Embedding(10375, 64, padding_idx=0)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (gru_layers): GRU(64, 128, bias=False, batch_first=True)
  (dense): Linear(in_features=128, out_features=64, bias=True)
  (loss_fct): CrossEntropyLoss()
)
Trainable parameters: 745984
GRU4Rec(
  (item_embedding): Embedding(10375, 64, padding_idx=0)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (gru_layers): GRU(64, 128, bias=False, batch_first=True)
  (dense): Linear(in_features=128, out_features=64, bias=True)
  (loss_fct): CrossEntropyLoss()
)
Trainable parameters: 745984
GRU4Rec(
  (item_embedding): Embedding(10375, 64, padding_idx=0)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (gru_layers): GRU(64, 128, bias=False, batch_first=True)
  (dense): Linear(in_features=128, out_features=64, bias=True)
  (loss_fct): CrossEntropyLoss()
)
Trainable parameters: 745984
epoch 0 training [time: 2.82s, train loss: 946.5942]
epoch 0 training [time: 2.82s, train loss: 946.5942]


### 4. Create recommendation result from trained model

In [82]:
external_user_ids = dataset.id2token(
    dataset.uid_field, list(range(dataset.user_num)))[1:]#fist element in array is 'PAD'(default of Recbole) ->remove it 

In [83]:
import torch
from recbole.data.interaction import Interaction

def add_last_item(old_interaction, last_item_id, max_len=50):
    new_seq_items = old_interaction['item_id_list'][-1]
    if old_interaction['item_length'][-1].item() < max_len:
        new_seq_items[old_interaction['item_length'][-1].item()] = last_item_id
    else:
        new_seq_items = torch.roll(new_seq_items, -1)
        new_seq_items[-1] = last_item_id
    return new_seq_items.view(1, len(new_seq_items))

def predict_for_all_item(external_user_id, dataset, model):
    model.eval()
    with torch.no_grad():
        uid_series = dataset.token2id(dataset.uid_field, [external_user_id])
        index = np.isin(dataset[dataset.uid_field].numpy(), uid_series)
        input_interaction = dataset[index]
        test = {
            'item_id_list': add_last_item(input_interaction, 
                                          input_interaction['item_id'][-1].item(), model.max_seq_length),
            'item_length': torch.tensor(
                [input_interaction['item_length'][-1].item() + 1
                 if input_interaction['item_length'][-1].item() < model.max_seq_length else model.max_seq_length])
        }
        new_inter = Interaction(test)
        new_inter = new_inter.to(config['device'])
        new_scores = model.full_sort_predict(new_inter)
        new_scores = new_scores.view(-1, test_data.dataset.item_num)
        new_scores[:, 0] = -np.inf  # set scores of [pad] to -inf
    return torch.topk(new_scores, 10)

In [85]:
topk_items = []
for external_user_id in external_user_ids:
    _, topk_iid_list = predict_for_all_item(external_user_id, dataset, model)
    last_topk_iid_list = topk_iid_list[-1]
    external_item_list = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()
    topk_items.append(external_item_list)
print(len(topk_items))

11031


In [86]:
external_item_str = [' '.join(x) for x in topk_items]
result = pd.DataFrame(external_user_ids, columns=['visitorid'])
result['prediction'] = external_item_str
result.head()

,visitorid,prediction
0,125625,400946 120098 141240 88098 240558 428556 75555...
1,361387,43485 347974 33907 116624 33912 185677 79484 4...
2,21004,181142 202170 306297 213102 6605 178451 3914 2...
3,712443,120098 239058 88098 204808 228180 240558 39888...
4,492414,174538 174284 139287 387318 142624 345095 3868...


In [87]:
input_features = test_data.dataset[np.isin(test_data.dataset[test_data.dataset.uid_field].numpy(), [2])]
input_features

The batch_size of interaction: 1
    user_id, torch.Size([1]), cpu, torch.int64
    item_id, torch.Size([1]), cpu, torch.int64
    timestamp, torch.Size([1]), cpu, torch.float32
    item_length, torch.Size([1]), cpu, torch.int64
    item_id_list, torch.Size([1, 50]), cpu, torch.int64
    timestamp_list, torch.Size([1, 50]), cpu, torch.float32


In [88]:
input_features['item_id_list']

tensor([[   2,    2,    2,    2,    2,    2,    2,    2, 1695,    2, 1708,    2,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0]])

In [89]:
model.eval()

GRU4Rec(
  (item_embedding): Embedding(10375, 64, padding_idx=0)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (gru_layers): GRU(64, 128, bias=False, batch_first=True)
  (dense): Linear(in_features=128, out_features=64, bias=True)
  (loss_fct): CrossEntropyLoss()
)

In [90]:
input_features['item_id_list'].shape

torch.Size([1, 50])

In [95]:
df[df["visitorid"]==361387]

,timestamp,visitorid,event,itemid,transactionid
35,1433224644638,361387,view,43485,NaN
8028,1433224068625,361387,view,43485,NaN
12450,1433224135170,361387,view,43485,NaN
28966,1433226471227,361387,view,43485,NaN
29134,1433224888005,361387,view,43485,NaN
32131,1433292711472,361387,view,352541,NaN
36349,1433226617310,361387,view,116475,NaN
38645,1433291978471,361387,view,43485,NaN
38698,1433292717627,361387,view,43485,NaN
44006,1433225173481,361387,view,43485,NaN


In [96]:
result["prediction"][1]

'43485 347974 33907 116624 33912 185677 79484 457231 421277 357583'

In [93]:
del external_item_str
del topk_items
del external_user_ids
del train_data
del valid_data
del test_data
del model
del Trainer
del logger
gc.collect()

211

In [94]:
del dataset
gc.collect()

23